> IMPORTS


In [2]:
# Manipulação de Arquivos e Sistema
import os
import sys
from tqdm.notebook import tqdm # Versão especial do tqdm para o Jupyter

# Visão Computacional e Imagem
import cv2
import PIL.Image
from PIL import Image

# Machine Learning e Deep Learning (PyTorch)
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Biometria e Detecção Facial
import mediapipe as mp

# Processamento de Dados e Visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Verificação de Versões e Hardware
print(f"Versão do Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"OpenCV: {cv2.__version__}")
print(f"MediaPipe: {mp.__version__}")

Versão do Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
PyTorch: 2.5.1
OpenCV: 4.10.0
MediaPipe: 0.10.31


> YOLO dataset training


In [3]:
from ultralytics import YOLO # Para detectar caminhões, prédios, etc.

model_yolo = YOLO('yolov8n.pt') # Modelo leve que identifica 80 tipos de objetos

def analisar_contexto(frame):
    results = model_yolo(frame)
    return results

> Teste de video teclado/mouse


In [11]:
import cv2
from ultralytics import YOLO

# Carrega o modelo 
model = YOLO('yolov8n.pt') 

# Caminho do vídeo
# Use o caminho exato que aparece no seu VS Code
video_path = 'video_test/teste_perifericos.mp4'

# Abre o vídeo
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo! Verifique o caminho.")

while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        # Roda a detecção (stream=True é bom para vídeos longos)
        results = model(frame, stream=True)
        
        for r in results:
            # r.plot() cria a imagem com as caixinhas e nomes (mouse, keyboard)
            annotated_frame = r.plot() 
            
            # Mostra a janela
            cv2.imshow("IA Detectando Objetos Reais", annotated_frame)

        # O cv2.waitKey(1) é essencial para a janela não travar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        # Fim do vídeo
        break

cap.release()
cv2.destroyAllWindows()


0: 640x384 1 keyboard, 52.2ms
Speed: 14.0ms preprocess, 52.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 50.8ms
Speed: 2.7ms preprocess, 50.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.2ms
Speed: 2.4ms preprocess, 44.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 43.1ms
Speed: 2.6ms preprocess, 43.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 60.1ms
Speed: 2.6ms preprocess, 60.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.6ms
Speed: 2.9ms preprocess, 44.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 44.1ms
Speed: 2.3ms preprocess, 44.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 keyboard, 1 cell phone, 46.0ms
Speed: 2.7ms preprocess, 46.0ms inference, 0.9ms postproce

- Com a detecção feita com sucesso, esta na hora de tentar identificar anomalias em videos para detectar se é feito de IA ou não.


In [18]:
video_path = 'video_test/teste_perifericos.mp4'
cap = cv2.VideoCapture(video_path)

# Pega o total de frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
alertas_encontrados = [] # Lista para guardar as bizarrices detectadas

print(f"Iniciando análise de {total_frames} frames...")

while cap.isOpened():
    success, frame = cap.read()
    
    if not success:
        print("Fim do vídeo ou erro na leitura.")
        break

    # Roda a detecção
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()
    
    # LÓGICA DE DETECÇÃO DE IA/ANOMALIAS
    analise_suspeita = False
    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Se a confiança for baixa (comum em vídeos de IA instáveis)
            if conf < 0.80:
                analise_suspeita = True
                msg = f"Frame {frame_count}: Forma instável detectada ({nome})"
                if msg not in alertas_encontrados:
                    alertas_encontrados.append(msg)
                    print(f"{msg}")

    # Feedback Visual para o usuário
    status_cor = (0, 0, 255) if analise_suspeita else (0, 255, 0) # Vermelho se suspeito, Verde se ok
    status_txt = "ANALISE: SUSPEITA" if analise_suspeita else "ANALISE: REAL"
    
    cv2.putText(annotated_frame, status_txt, (50, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, status_cor, 3)
    
    # Contador de frames
    frame_count += 1
    cv2.putText(annotated_frame, f"Frame: {frame_count}/{total_frames}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Analisando Realismo", annotated_frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# LIMPEZA
cap.release()
cv2.destroyAllWindows()
for i in range(10):
    cv2.waitKey(1)

print("\n--- RESUMO DA ANALISE ---")
if alertas_encontrados:
    print(f"Foram detectadas {len(alertas_encontrados)} inconsistências no vídeo.")
else:
    print("Nenhuma inconsistência grave detectada. O vídeo parece real.")

Iniciando análise de 139 frames...
Frame 0: Forma instável detectada (keyboard)
Frame 1: Forma instável detectada (keyboard)
Frame 2: Forma instável detectada (keyboard)
Frame 3: Forma instável detectada (keyboard)
Frame 6: Forma instável detectada (keyboard)
Frame 7: Forma instável detectada (cell phone)
Frame 8: Forma instável detectada (cell phone)
Frame 9: Forma instável detectada (cell phone)
Frame 10: Forma instável detectada (cell phone)
Frame 62: Forma instável detectada (keyboard)
Frame 63: Forma instável detectada (keyboard)
Frame 79: Forma instável detectada (tv)
Frame 80: Forma instável detectada (tv)
Frame 80: Forma instável detectada (tie)
Frame 81: Forma instável detectada (tie)
Frame 81: Forma instável detectada (person)
Frame 82: Forma instável detectada (cell phone)
Frame 82: Forma instável detectada (person)
Frame 82: Forma instável detectada (tie)
Frame 83: Forma instável detectada (tie)
Frame 93: Forma instável detectada (cell phone)
Frame 94: Forma instável detect

---

> teste do detector com video de IA Gemini


In [16]:
video_path = 'video_test/Teclado_Vira_Mouse_Vídeo_Pronto.mp4'
cap = cv2.VideoCapture(video_path)

# Pega o total de frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
alertas_encontrados = [] # Lista para guardar as bizarrices detectadas

print(f"Iniciando análise de {total_frames} frames...")

while cap.isOpened():
    success, frame = cap.read()
    
    if not success:
        print("Fim do vídeo ou erro na leitura.")
        break

    # Roda a detecção
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()
    
    # LÓGICA DE DETECÇÃO DE IA/ANOMALIAS
    analise_suspeita = False
    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Se a confiança for baixa (comum em vídeos de IA instáveis)
            if conf < 0.80:
                analise_suspeita = True
                msg = f"Frame {frame_count}: Forma instável detectada ({nome})"
                if msg not in alertas_encontrados:
                    alertas_encontrados.append(msg)
                    print(f"{msg}")

    # Feedback Visual para o usuário
    status_cor = (0, 0, 255) if analise_suspeita else (0, 255, 0) # Vermelho se suspeito, Verde se ok
    status_txt = "ANALISE: SUSPEITA" if analise_suspeita else "ANALISE: REAL"
    
    cv2.putText(annotated_frame, status_txt, (50, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, status_cor, 3)
    
    # Contador de frames
    frame_count += 1
    cv2.putText(annotated_frame, f"Frame: {frame_count}/{total_frames}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Analisando Realismo", annotated_frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# LIMPEZA
cap.release()
cv2.destroyAllWindows()
for i in range(10):
    cv2.waitKey(1)

print("\n--- RESUMO DA ANALISE ---")
if alertas_encontrados:
    print(f"Foram detectadas {len(alertas_encontrados)} inconsistências no vídeo.")
else:
    print("Nenhuma inconsistência grave detectada. O vídeo parece real.")

Iniciando análise de 192 frames...
Frame 66: Forma instável detectada (keyboard)
Frame 69: Forma instável detectada (keyboard)
Frame 70: Forma instável detectada (keyboard)
Frame 71: Forma instável detectada (keyboard)
Frame 72: Forma instável detectada (keyboard)
Frame 73: Forma instável detectada (keyboard)
Frame 74: Forma instável detectada (keyboard)
Frame 75: Forma instável detectada (keyboard)
Frame 76: Forma instável detectada (cat)
Frame 77: Forma instável detectada (cat)
Frame 78: Forma instável detectada (donut)
Frame 85: Forma instável detectada (cat)
Frame 86: Forma instável detectada (cat)
Frame 87: Forma instável detectada (cat)
Frame 88: Forma instável detectada (cat)
Frame 89: Forma instável detectada (cat)
Frame 97: Forma instável detectada (cat)
Frame 100: Forma instável detectada (bed)
Frame 101: Forma instável detectada (cat)
Frame 102: Forma instável detectada (cat)
Frame 103: Forma instável detectada (cat)
Frame 104: Forma instável detectada (cat)
Frame 105: Forma

Decepcionante, mas promissor...

Conseguimos identificar em algum momento coisas suspeitas no video, mas isso nao mudou o criterio, preciso de algo mais preciso e analitico sobre os videos.


-- CODIGO COM MEMORIA


In [15]:
video_path = 'video_test/Teclado_Vira_Mouse_Vídeo_Pronto.mp4'
cap = cv2.VideoCapture(video_path)

# Pega o total de frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
alertas_encontrados = [] # Lista para guardar as bizarrices detectadas

print(f"Iniciando análise de {total_frames} frames...")

historico_objeto = None

while cap.isOpened():
    success, frame = cap.read()
    if not success: break

    results = model(frame, verbose=False)
    
    for r in results:
        for box in r.boxes:
            nome_atual = model.names[int(box.cls[0])]
            
            # LOGICA DE MUDANÇA BRUSCA:
            if historico_objeto and nome_atual != historico_objeto:
                print(f"ALERTA: Objeto mudou de {historico_objeto} para {nome_atual} instantaneamente!")
                status_txt = "ANALISE: IA DETECTADA (Transformacao)"
                status_cor = (0, 0, 255)
            else:
                status_txt = "ANALISE: REAL"
                status_cor = (0, 255, 0)
            
            historico_objeto = nome_atual # Salva para o próximo frame
            
    success, frame = cap.read()
    
    if not success:
        print("Fim do vídeo ou erro na leitura.")
        break

    # Roda a detecção
    results = model(frame, verbose=False)
    annotated_frame = results[0].plot()
    
    # LÓGICA DE DETECÇÃO DE IA/ANOMALIAS
    analise_suspeita = False
    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Se a confiança for baixa (comum em vídeos de IA instáveis)
            if conf < 0.80:
                analise_suspeita = True
                msg = f"Frame {frame_count}: Forma instável detectada ({nome})"
                if msg not in alertas_encontrados:
                    alertas_encontrados.append(msg)
                    print(f"{msg}")

    # Feedback Visual para o usuário
    status_cor = (0, 0, 255) if analise_suspeita else (0, 255, 0) # Vermelho se suspeito, Verde se ok
    status_txt = "ANALISE: SUSPEITA" if analise_suspeita else "ANALISE: REAL"
    
    cv2.putText(annotated_frame, status_txt, (50, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, status_cor, 3)
    
    # Contador de frames
    frame_count += 1
    cv2.putText(annotated_frame, f"Frame: {frame_count}/{total_frames}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Analisando Realismo", annotated_frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# LIMPEZA
cap.release()
cv2.destroyAllWindows()
for i in range(10):
    cv2.waitKey(1)

print("\n--- RESUMO DA ANALISE ---")
if alertas_encontrados:
    print(f"Foram detectadas {len(alertas_encontrados)} inconsistências no vídeo.")
else:
    print("Nenhuma inconsistência grave detectada. O vídeo parece real.")

Iniciando análise de 192 frames...
Frame 34: Forma instável detectada (keyboard)
Frame 35: Forma instável detectada (keyboard)
Frame 36: Forma instável detectada (keyboard)
Frame 37: Forma instável detectada (keyboard)
ALERTA: Objeto mudou de keyboard para cat instantaneamente!
Frame 38: Forma instável detectada (cat)
ALERTA: Objeto mudou de cat para donut instantaneamente!
ALERTA: Objeto mudou de donut para cat instantaneamente!
Frame 42: Forma instável detectada (cat)
Frame 43: Forma instável detectada (cat)
Frame 44: Forma instável detectada (cat)
Frame 48: Forma instável detectada (cat)
ALERTA: Objeto mudou de cat para bed instantaneamente!
Frame 50: Forma instável detectada (cat)
ALERTA: Objeto mudou de bed para cat instantaneamente!
Frame 51: Forma instável detectada (cat)
Frame 52: Forma instável detectada (cat)
Frame 53: Forma instável detectada (cat)
Frame 55: Forma instável detectada (cat)
Frame 57: Forma instável detectada (cat)
Frame 58: Forma instável detectada (cat)
Frame

In [22]:
from mss import mss
from plyer import notification
import time

# Configuração da captura de tela
sct = mss()
# Define a região da tela
monitor = sct.monitors[1] 

notificacao_enviada = False
ultimo_alerta_time = 0
tempo_de_espera_alerta = 60 # Espera 60 segundos para avisar de novo (evita spam)

print("Monitoramento em tempo real iniciado... Pressione 'q' para parar.")

while True:
    # Captura o frame da tela
    sct_img = sct.grab(monitor)
    frame = np.array(sct_img)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR) # Converte para o formato do OpenCV

    # IA analisa a imagem da tela
    results = model(frame, verbose=False, conf=0.8) # Confiança mínima 0.8
    
    analise_suspeita = False
    motivo = ""

    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Lógica de suspeita (confiança baixa ou objetos instáveis)
            if conf < 0.55: 
                analise_suspeita = True
                motivo = f"Detectado objeto instável: {nome}"
                break

    # Lógica de Notificação Única
    current_time = time.time()
    if analise_suspeita and (current_time - ultimo_alerta_time > tempo_de_espera_alerta):
        notification.notify(
            title='Alerta de Segurança de IA',
            message=f'Detectamos uma possível manipulação visual na sua tela: {motivo}',
            app_name='Detector Protetor',
            timeout=10 # Tempo que a notificação fica na tela
        )
        print(f"NOTIFICAÇÃO ENVIADA: {motivo}")
        ultimo_alerta_time = current_time

    # Mostra o que a IA está vendo
    annotated_frame = results[0].plot()
    cv2.imshow("Monitor de IA em Tempo Real", cv2.resize(annotated_frame, (800, 450)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
print("Monitoramento encerrado.")

Monitoramento em tempo real iniciado... Pressione 'q' para parar.
Monitoramento encerrado.


In [6]:
import cv2
import numpy as np
from mss import mss
from plyer import notification
import time
from ultralytics import YOLO

# Carrega o modelo (certifique-se de que o model já foi definido antes)
model = YOLO('yolov8n.pt') 

sct = mss()
monitor = sct.monitors[1] 

# Variáveis de controle de notificação
ultimo_alerta_time = 0
tempo_de_espera_alerta = 60 

print("Monitoramento em tempo real iniciado.")

while True:
    sct_img = sct.grab(monitor)
    frame = np.array(sct_img)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

    # IA analisa a tela
    results = model(frame, verbose=False, conf=0.6)
    
    analise_suspeita = False
    motivo = ""

    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            nome = model.names[int(box.cls[0])]
            
            # Se a confiança for baixa, marcamos como suspeito
            if conf < 0.8: 
                analise_suspeita = True
                motivo = f"Objeto com forma indefinida ({nome})"
                break

    # ENVIO DA NOTIFICAÇÃO NATIVA
    current_time = time.time()
    if analise_suspeita and (current_time - ultimo_alerta_time > tempo_de_espera_alerta):
        try:
            notification.notify(
                title='Cuidado: Vídeo Suspeito',
                message=f'A IA detectou algo estranho na sua tela: {motivo}. Pode ser um vídeo falso.',
                app_name='Detector Protetor',
                # Você pode colocar um ícone .ico aqui para parecer mais profissional
                # ticker='Alerta de Segurança', 
                timeout=10
            )
            print(f"Notificação enviada para a Central do Windows: {motivo}")
            ultimo_alerta_time = current_time
        except Exception as e:
            print(f"Erro ao enviar notificação: {e}")

    # Visualização (aperte Q para sair)
    annotated_frame = results[0].plot()
    cv2.imshow("Monitor Protetor", cv2.resize(annotated_frame, (640, 360)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

Monitoramento em tempo real iniciado.
Notificação enviada para a Central do Windows: Objeto com forma indefinida (cat)
